# Singular Value Decomposition (CF)

CF в любом из вариантов (**user-based** / **item-based**) имеет проблемы, если матрица сильно разрежена, а также имеет проблему масштабируемости. Это не позволяет использовать решения на основе CF на очень больших данных. 

Описание проблемы разреженности  [Sarwar et al. (2000)](http://files.grouplens.org/papers/webKDD00.pdf), где предлагается *Singular Value Decomposition (SVD)* для решения.

---

### Как?

SVD разкладывает матрицу размера $m\times n$ в матрицы $P$, $\Sigma$ и $Q$:

\begin{equation}
R = P\Sigma Q^{\top}.
\end{equation}

$P$ и $Q$ это ортогональные матрицы и $\Sigma$ диагональная матрица состоящая из сингулярных значений рейтингов в качестве диагональных значений ([Billsus and Pazzani, 1998](https://www.ics.uci.edu/~pazzani/Publications/MLC98.pdf), [Sarwar et al. (2000)](http://files.grouplens.org/papers/webKDD00.pdf)).

![](img/svd.png)

Матрица рейтингов рассчитывается: 

\begin{equation}
R_k = P_k\Sigma_k Q_k^{\top}.
\end{equation}


---

### SVD 

> 1. Нормализованная матрица $R_{norm}$ раскладывается на $P$, $\Sigma$ и $Q$
> 2. Уменьшаем $\Sigma$ до размерности $k$ и трансформируем в $\Sigma_k$
> 3. Считаем квадратный корень из $\Sigma_k$ для получения $\Sigma_k^{\frac{1}{2}}$
> 4. Считаем финальную матрицу $P_k\Sigma_k^{\frac{1}{2}}$ и $\Sigma_k^{\frac{1}{2}}Q_k^{\top}$, которая будет использоваться для расчета рекомендаций.

---


In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix

import pandas as pd
import numpy as np
import zipfile

import os
import sys

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [4]:
pd.crosstab(ratings.userId, ratings.movieId, ratings.rating, aggfunc=sum)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def rating_matrix(ratings):
    """
    1. Запоним NaN средним рейтингом
    2. Нормализуем рейтинг относительно среднего значения
    
    :param ratings : DataFrame
    :return
        - R : Numpy array рейтингов
        - df : DataFrame рейтингов
    """
    
    # средний рейтинг
    umean = ratings.groupby(by='userId')['rating'].mean()

    # заполняем пустоты
    df = pd.crosstab(ratings.userId, ratings.movieId, ratings.rating, aggfunc=sum)
    df = df.fillna(df.mean(axis=0))

    # нормализация по среднему значению
    df = df.subtract(umean, axis=0)
    
    # в numpy
    R = df.to_numpy()
    
    return R, df

# записываем результат
R, df = rating_matrix(ratings)

$R$ готовая матрица для применения

In [6]:
df

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,1.322470,0.851869,0.611017,-0.165385,0.717857,1.334615,0.733019,1.250000,0.600000,0.900820,...,-0.050000,-2.050000,0.450000,-1.550000,-1.050000,2.450000,1.950000,2.450000,0.450000,2.450000
2,0.385628,-0.084973,-0.325825,-1.102227,-0.218985,0.397773,-0.203823,0.313158,-0.336842,0.513158,...,-0.986842,-2.986842,-0.486842,-2.486842,-1.986842,1.513158,1.013158,1.513158,-0.486842,1.513158
3,0.303842,-0.166758,-0.407611,-1.184012,-0.300770,0.315988,-0.285609,0.231373,-0.418627,-0.117808,...,-1.068627,-3.068627,-0.568627,-2.568627,-2.068627,1.431373,0.931373,1.431373,-0.568627,1.431373
4,-0.475570,-0.946170,-1.187022,-1.963424,-1.080182,-0.463424,-1.065020,-0.548039,-1.198039,-0.348039,...,-1.848039,-3.848039,-1.348039,-3.348039,-2.848039,0.651961,0.151961,0.651961,-1.348039,0.651961
5,-0.037530,-0.508131,0.090000,-1.525385,-0.642143,-0.025385,-0.626981,-0.110000,-0.760000,-0.459180,...,-1.410000,-3.410000,-0.910000,-2.910000,-2.410000,1.090000,0.590000,1.090000,-0.910000,1.090000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.225411,-0.245190,-0.486042,-1.262443,-0.379202,0.352941,-0.364040,0.152941,-0.497059,-0.196239,...,-1.147059,-3.147059,-0.647059,-2.647059,-2.147059,1.352941,0.852941,1.352941,-0.647059,1.352941
668,0.122470,-0.348131,-0.588983,-1.365385,-0.482143,0.134615,-0.466981,0.050000,-0.600000,-0.299180,...,-1.250000,-3.250000,-0.750000,-2.750000,-2.250000,1.250000,0.750000,1.250000,-0.750000,1.250000
669,0.521118,0.050518,-0.190334,-0.966736,-0.083494,0.533264,-0.068332,0.448649,-0.201351,0.099468,...,-0.851351,-2.851351,-0.351351,-2.351351,-1.851351,1.648649,1.148649,1.648649,-0.351351,1.648649


### SVD


1. ```fit()``` - расчет SVD рейтингов и сохранение матриц P, S, Q
2. ```predict()``` - матрицы P, S и Qh для создания предикта по пользователю - элементу. Учитывая, что мы сделали вычитание фактического рейтинга из среднего по пользователю, нам необходимо будет вернуть значение, прибавив предикт к среднему рейтингу
3. ```recommend()``` - функция рекомендаций

In [8]:
class SVD:
    
    def __init__(self, umeam):
        """
        :param
            - umean : среднее значение рейтингов по пользователю
        """
        self.umean = umean.to_numpy()
        
        # init svd 
        self.P = np.array([])
        self.S = np.array([])
        self.Qh = np.array([])
        
        # init пользователь и элемент
        self.u_factors = np.array([])
        self.i_factors = np.array([])
    
    def fit(self, R):
        """
        Fit  SVD
        """
        P, s, Qh = np.linalg.svd(R, full_matrices=False)
        
        self.P = P
        self.S = np.diag(s)
        self.Qh = Qh
        
        # скрытые факторы по пользователю (u_factors) и по элементу (i_factors)
        self.u_factors = np.dot(self.P, np.sqrt(self.S))
        self.i_factors = np.dot(np.sqrt(self.S), self.Qh)
    
    def predict(self, userid, itemid):
        """
        Предикт по пользователю
        
        :param
            - userid : пользователь
            - itemid : элемент
            
        :return
            - r_hat : predicted rating
        """
        
        # предикт вычисляется по факторам пользователя и элемента
        r_hat = np.dot(self.u_factors[userid,:], self.i_factors[:,itemid])
        
        # суммируем со средним значением
        r_hat += self.umean[userid]
        
        return r_hat
        
    
    def recommend(self, userid):
        """
        :param
            - userid : id пользователя
        """
        
        # предикт для пользователя по факторам     
        predictions = np.dot(self.u_factors[userid,:], self.i_factors) + self.umean[userid]
        
        # сортировка результата
        top_idx = np.flip(np.argsort(predictions))
        preds = predictions[top_idx]
        
        return top_idx, preds
        

Создадим SVD модель

Передадим средний рейтинг, как базовый элемент

In [9]:
umean = ratings.groupby(by='userId')['rating'].mean()

# svd
svd = SVD(umean)

# fit
svd.fit(R)

### Предикт рейтинга

In [10]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [11]:
# для кого делаем предикт
userid = 1

# какие элементы подбираем
items = [1,3,6,47,50,70,101,110,151,157]

# формирование предикта
for itemid in items:
    r = svd.predict(userid=userid, itemid=itemid)
    print('предикт для пользователя ={} по элементу ={} : {}'.format(userid, itemid, r))

предикт для пользователя =1 по элементу =1 : 3.4018691588785046
предикт для пользователя =1 по элементу =3 : 2.384615384615384
предикт для пользователя =1 по элементу =6 : 3.2830188679245222
предикт для пользователя =1 по элементу =47 : 4.000000000000001
предикт для пользователя =1 по элементу =50 : 5.000000000000003
предикт для пользователя =1 по элементу =70 : 3.3333333333333375
предикт для пользователя =1 по элементу =101 : 4.224576271186442
предикт для пользователя =1 по элементу =110 : 3.0526315789473695
предикт для пользователя =1 по элементу =151 : 2.5729166666666656
предикт для пользователя =1 по элементу =157 : 4.700000000000005


### Создание рекомендаций

In [12]:
userid = 1

# сортировка предикта
top_indx, preds = svd.recommend(userid=userid)

rec_movies = movies[movies['movieId'].isin(top_indx)]

# списко элементов, которые пользователь отметил рейтингом
uitems = ratings.loc[ratings.userId == userid].movieId.to_list()

# убираем фильмы уже оцененные пользователем
top10 = np.setdiff1d(top_indx, uitems, assume_unique=True)[:10]

# топ - N
top10_idx = list(np.where(top_indx == idx)[0][0] for idx in top10)
top10_predictions = preds[top10_idx]

# добавляем название и жанр
zipped_top10 = list(zip(top10,top10_predictions))
top10 = pd.DataFrame(zipped_top10, columns=['movieId','predictions'])
List = pd.merge(top10, movies, on='movieId', how='inner')

List

,movieId,predictions,title,genres
0,494,5.0,Executive Decision (1996),Action|Adventure|Thriller
1,524,5.0,Rudy (1993),Drama
2,238,5.0,Far From Home: The Adventures of Yellow Dog (1...,Adventure|Children
3,16,5.0,Casino (1995),Crime|Drama
4,518,5.0,"Road to Wellville, The (1994)",Comedy
5,37,5.0,Across the Sea of Time (1995),Documentary|IMAX
6,237,5.0,Forget Paris (1995),Comedy|Romance
7,523,5.0,Ruby in Paradise (1993),Drama
8,195,5.0,Something to Talk About (1995),Comedy|Drama|Romance
9,129,5.0,Pie in the Sky (1996),Comedy|Romance
